# Fun with Pandas
To get comfortable with the the **Pandas** package, I will base my practice off exercises from [DataCamp](http://www.datacamp.com) in this section of the notebook. Specifically I will demonstrate how great **Pandas** is for Exploratory Data Analysis, Data Manipulation and more.



## Pandas Basics

## Imported Data with Pandas
We might even want to conduct a broader analysis of other baseball teams. The Pittsburgh Pirates team was kind enough to send us over a file of their 2018 team roster to take a look at and use in our study.
First thing we need to do is get that data into our environment quickly using the builtin file functions available in Python.

In [ ]:
# Load file as numpy array: np_pirates_roster
np_pirates_roster = np.recfromcsv('2018_PghPirates_Roster.csv')

# Print shape of np_pirates_roster
print(np.shape(np_pirates_roster))